# Import Libraries

In [1]:
!pip install keras-cv-attention-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.4/613.4 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.79.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires 

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mpl

from tqdm.notebook import tqdm
from multiprocessing import cpu_count
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
from keras_cv_attention_models import convnext


import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

import os
import time
import pickle
import math
import random
import sys
import cv2
import gc
import datetime

print(f'Tensorflow Version: {tf.__version__}')
print(f'Python Version: {sys.version}')

Tensorflow Version: 2.11.0
Python Version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]


In [3]:
now = datetime.datetime.now().strftime("%d-%b-%Y %H-%M-%S")
np.save(now, np.array([now]))

## MP policy

In [4]:
# # float32 or mixed_float16 (mixed precision: compute float16, variable float32)
# # TPU is fast enough and has enough memory to use float32
# policy = tf.keras.mixed_precision.Policy('float32')
# tf.keras.mixed_precision.set_global_policy(policy)

# print(f'Compute dtype: {tf.keras.mixed_precision.global_policy().compute_dtype}')
# print(f'Variable dtype: {tf.keras.mixed_precision.global_policy().variable_dtype}')

## Matplotlib Config¶


In [5]:
# MatplotLib Global Settings
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['axes.titlesize'] = 24

## Config

In [6]:
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)

## Load Data

In [7]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  '/kaggle/input/fake-images-detecttion-dataset/Keras_Data-20230318T161559Z-001/Keras_Data/Train_Data',
  seed=0,
  image_size=(160, 160),
  batch_size=16)

Found 47995 files belonging to 2 classes.


In [8]:
valid_ds = tf.keras.utils.image_dataset_from_directory(
  '/kaggle/input/fake-images-detecttion-dataset/Keras_Data-20230318T161559Z-001/Keras_Data/Validation_Data',
  seed=0,
  image_size=(160, 160),
  batch_size=8)

Found 6000 files belonging to 2 classes.


In [9]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  '/kaggle/input/fake-images-detecttion-dataset/Keras_Data-20230318T161559Z-001/Keras_Data/Test_Data',
  seed=0,
  image_size=(160, 160),
  batch_size=8)

Found 6001 files belonging to 2 classes.


In [10]:
# normalization_layer = tf.keras.layers.Rescaling(1./255)
# normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))


In [11]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = valid_ds.prefetch(buffer_size=AUTOTUNE)

# Model

In [12]:
# Seed all random number generators
def seed_everything(seed=0):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything()


In [13]:
def normalize(image):
#     # Repeat channels to create 3 channel images required by pretrained ConvNextV2 models
#     image = tf.repeat(image, repeats=3, axis=3)
    # Cast to float 32
    image = tf.cast(image, tf.float32)
    # Normalize with respect to ImageNet mean/std
    image = tf.keras.applications.imagenet_utils.preprocess_input(image, mode='torch')

    return image

In [14]:
STRATEGY = tf.distribute.MirroredStrategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

N_REPLICAS = STRATEGY.num_replicas_in_sync

In [15]:
N_REPLICAS

1

In [16]:
from keras_cv_attention_models import mobilenetv3

In [17]:
def get_model(path):
    # Inputs, note the names are equal to the dictionary keys in the dataset
    image = tf.keras.layers.Input((160, 160, 3), name='image', dtype=tf.uint8)

    # Normalize Input
    image_norm = normalize(image)

    # CNN Prediction in range [0,1]
    x = convnext.ConvNeXtV2Tiny(
        input_shape=(160, 160, 3),
        pretrained='imagenet',
        num_classes=0,
    )(image_norm)

    # Average Pooling BxHxWxC -> BxC
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    # Dropout to prevent Overfitting
    x = tf.keras.layers.Dropout(0.1)(x)
    # Output value between [0, 1] using Sigmoid function
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Define model with inputs and outputs
    model = tf.keras.models.Model(inputs=image, outputs=outputs)

    # Load pretrained Model Weights
    model.load_weights(path)

    # Set model non-trainable
    model.trainable = False

    # Compile model
    model.compile()

    return model

In [18]:
model = get_model('/kaggle/input/deep-fake-models-trial1/covnext_tiny.h5')

    

115145456/115145456 [==============================] - 2s 0us/step
>>>> Load pretrained from: /root/.keras/models/convnext_v2_tiny_imagenet.h5


# Test

In [19]:
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [20]:
y_pred = []  # store predicted labels
y_true = []  # store true labels

# iterate over the dataset
for image_batch, label_batch in test_ds:   # use dataset.unbatch() with repeat
   # append true labels
   y_true.append(label_batch)
   # compute predictions
   preds = model.predict(image_batch)
   # append predicted labels
   preds[preds <= 0.5] = 0.
   preds[preds > 0.5] = 1.
   y_pred.append(preds)

# convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)

1/1 [==============================] - 0s 71ms/step


In [21]:
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn import confusion_matrix

# cm = confusion_matrix(correct_labels.numpy(), predicted_labels.numpy())

In [22]:
# cm

In [23]:
from sklearn.metrics import classification_report
print(classification_report(correct_labels.numpy(), predicted_labels.numpy()))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      3001
           1       0.97      0.97      0.97      3000

    accuracy                           0.97      6001
   macro avg       0.97      0.97      0.97      6001
weighted avg       0.97      0.97      0.97      6001

